<a href="https://colab.research.google.com/github/HisameOgasahara/HF_transformers/blob/main/HFHT_7_3_%EC%96%91%EC%9E%90%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install \
     scikit-learn==1.4.2 \
     transformers==4.43.0 \
     datasets==2.20.0 \
     peft==0.10.0 \
     accelerate==0.32.1 \
     bitsandbytes==0.43.1 \
     trl==0.9.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 

양자화: Mapping 함수를 의식해야. Fp32의 수 범위를 Fp16/8/4의 범위(혹은 이들의 변환이나 분포를 고려해)로 매핑.

# 7.3.1 bitsandbytes

### 8-bit

In [ ]:
# 양자화 미사용
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "nlpai-lab/ko-gemma-2b-v1",
    device_map="auto"
)

In [2]:
from transformers import AutoModelForCausalLM

model_8bit = AutoModelForCausalLM.from_pretrained(
    "nlpai-lab/ko-gemma-2b-v1",
    device_map="auto",
    load_in_8bit=True,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


RuntimeError: No GPU found. A GPU is needed for quantization.

오프로딩: 8비트 양자화된 모델을 GPU에서 실행하는 동안, 정밀도가 중요한 특정 연산은 CPU로 "오프로드"하여 더 높은 정밀도로 계산하고, 그 결과를 다시 GPU로 가져와서 나머지 연산

In [ ]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(llm_int8_enable_fp32_cpu_offload=True)
model_8bit = AutoModelForCausalLM.from_pretrained(
    "nlpai-lab/ko-gemma-2b-v1",
    device_map="auto",
    quantization_config=quantization_config,
)

이상치 임계값

In [ ]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(llm_int8_threshold=10.0) # 이 옵션은 8비트 양자화 과정에서 이상치(outlier)를 처리하는 방식을 결정하는 임계값(threshold)을 설정합니다.
model_8bit = AutoModelForCausalLM.from_pretrained(
    "nlpai-lab/ko-gemma-2b-v1",
    device_map="auto",
    quantization_config=quantization_config,
)

# BitsAndBytes는 동적 양자화(dynamic quantization)를 사용하는데, 활성화 값(activation)의 범위를 보고 양자화 파라미터를 결정합니다. llm_int8_threshold는 이 범위 밖의 값(이상치)들을 어떻게 처리할지 결정합니다.
# 설정된 임계값보다 큰 절댓값을 가진 활성화 값들은 fp32로 처리하고, 임계값 이하의 값들은 int8로 양자화

모듈 변환 건너뛰기

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(llm_int8_skip_modules=["lm_head"]) #지정된 모듈(lm_head)은 양자화하지 않고 건너뛰도록(skip) 설정하는 옵션. 언어 모델의 가장 마지막 레이어로 확률분포를 출력하는 레이어를 양자화 하지 않고 skip. 이 부분은 텍스트 생성에 영향을 크게 미쳐서 양자화 X
model_8bit = AutoModelForCausalLM.from_pretrained(
    "nlpai-lab/ko-gemma-2b-v1",
    device_map="auto",
    quantization_config=quantization_config,
)

### 4-bit

데이터 타입 계산하기

In [ ]:
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="bfloat16", #  4비트 양자화된 모델을 사용할 때, 계산(compute)에 사용할 데이터 타입(dtype)을 "bfloat16"으로 지정하는 옵션. float32보다 정밀도는 낮지만, 속도가 빠르고 메모리 사용량이 적습니다.
)                                      #  float16은 bfloat16보다 표현 범위가 좁아서, 매우 크거나 작은 값을 표현할 때 오버플로우(overflow)나 언더플로우(underflow)가 발생할 수 있습니다.
model_4bit = AutoModelForCausalLM.from_pretrained(
    "nlpai-lab/ko-gemma-2b-v1",
    device_map="auto",
    quantization_config=quantization_config,
)

#####주의: 4비트는 모델의 웨이트가 4비트인거고, 16비트는 계산시에 4비트를 16비트 웨이트를 변환해 계산한다는 의미.
# load_in_4bit=True: 모델의 가중치(weights) 를 4비트로 양자화하여 저장 합니다. 이렇게 하면 모델의 메모리 사용량이 크게 줄어듭니다. (저장할때는 4bit를 사용)
# bnb_4bit_compute_dtype="bfloat16": 양자화된 4비트 가중치를 사용하여 계산(예: 행렬 곱셈)을 할 때는, 이 4비트 가중치를 bfloat16 (또는 float32, float16)으로 변환(dequantize)하여 계산을 수행합니다. (계산 할때는 16bit(bfloat16)를 사용)

NF4(Normal Float 4)

In [ ]:
from transformers import BitsAndBytesConfig

nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4", #"nf4" (4-bit NormalFloat): QLoRA 논문에서 제안된 방식으로, 가중치의 분포가 정규 분포(normal distribution)를 따른다는 가정 하에 최적화된 양자화 방식. 모델의 가중치가 분포가 정규분포에 가까울 수록 fp4에 비해서 더 높은 정확도
)
model_nf4 = AutoModelForCausalLM.from_pretrained(
    "nlpai-lab/ko-gemma-2b-v1",
    device_map="auto",
    quantization_config=nf4_config,
)

중첩 양자화

In [ ]:
from transformers import BitsAndBytesConfig

double_quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
)
model_double_quant = AutoModelForCausalLM.from_pretrained(
    "nlpai-lab/ko-gemma-2b-v1",
    device_map="auto",
    quantization_config=double_quant_config,
)

기존 4비트 양자화 (without double quantization):

블록 분할: 가중치 행렬을 작은 블록(block)으로 나눕니다 (예: 블록 크기 = 64).

양자화 상수: 각 블록마다, 해당 블록의 가중치 범위를 나타내는 양자화 상수 c (normalization constant)를 계산합니다. 이 상수는 일반적으로 float32 값입니다.

4비트 양자화: 각 블록 내의 가중치를 4비트로 양자화합니다.



Double Quantization (이중 양자화):

1단계 양자화 (First quantization): 기존 4비트 양자화와 동일하게, 각 블록의 가중치를 4비트로 양자화하고, 각 블록에 대한 float32 양자화 상수 c1 (첫 번째 양자화 상수)를 계산합니다.

2단계 양자화 (Second quantization):

1단계에서 얻은 float32 양자화 상수들 c1을 모아서, 이들을 다시 양자화합니다. 일반적으로 8비트 양자화(int8 또는 float8)를 사용합니다.

이 과정에서 또 다른 양자화 상수 c2 (두 번째 양자화 상수, meta-quantization constant)가 계산됩니다. c2는 float32 입니다.

QLoRA 논문에 따르면 성능에는 거의 영향을 미치지 않습니다.